In [ ]:
# 数据爬取及预处理

In [2]:
import requests
proxy_url = 'http://localhost:5000/random'
proxies = {'http': 'http://' + requests.get(proxy_url).text.strip()}
print(proxies)

ConnectionError: HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /random (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002D5E9C9A8B0>: Failed to establish a new connection: [WinError 10061] 由于目标计算机积极拒绝，无法连接。'))

In [1]:
import nest_asyncio
import time
import os
import asyncio
import random
import requests
nest_asyncio.apply()

ids=['2372300','100003233502','100005011423','62197285636']
start_page=0
end_page=101
sleep_time=3
level_list=[0,1,2,3]

url='http://club.jd.com/comment/productPageComments.action'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/82.0.4051.0 Safari/537.36 Edg/82.0.425.0',
      'referer':'https://item.jd.com/',
      'Cookie':'__jda=122270672.16583767074391699399866.1658376707.1659773697.1659777991.7; unpl=JF8EAMhnNSttUElRUk5RSBQXHF1QWw4ASEdQbGYFB1pdG1wGHwceGhN7XlVdXhRKHh9uYBRUX1NPXQ4bAisSEXteXVdZDEsWC2tXVgQFDQ8VXURJQlZAFDNVCV9dSRZRZjJWBFtdT1xWSAYYRRMfDlAKDlhCR1FpMjVkXlh7VAQrAhwWGUlbU1hcDk4VC29gDVxfUENdAhwyKxUge21SWFQOSicCX2Y1FgkETlIAGwUaXxBMWV1cWw9NFgVqZQ1UWlBDVg0TCxwVIEptVw; __jdu=16583767074391699399866; __jdc=122270672; __jdv=122270672%7Candroidapp%7Ct_335139774%7Cappshare%7CCopyURL%7C1659777992090; areaId=6; ipLoc-djd=6-318-44144-60003F326B83FA39581E22DECBEB0AB7715E30ADCCC2F68CBC25EDB8EE53525351E0E9EA98AFABA76E88E4047E080448C244A1B0B64935EC2F13D9B0DDF74EC571BEBF63EDE605298A7E4EAD85924A931959CC88C35811E4BE9A75E1DE9; pinId=RdN1sMVryvY3jRnzBDNjpLV9-x-f3wj7; pin=jd_743a7486a59a8; unick=jd_173022gsy; ceshi3.com=000; _tp=X17pPm3xiI5ewVPdIJ8yvmpfTPdGemmpmhewCcj6Eg8%3D; _pst=jd_743a7486a59a8; cn=2; __jd_ref_cls=MDownLoadFloat_AppArouseA1; 3AB9D23F7A4B3C9B=GXPMKWVT3FPEJJ23VAPVNS57IMPPJAQM4C2MWSMAQKJ6P2IOHZTQUWXTUZE764KJBSMJ4ZHBL4FN4RCPR4BRE63JEA',
      'content_type':'application/json'
      }

def get_proxy():
    url = 'http://http://www.zdopen.com/ShortProxy/GetIP/?api=202207291010568666&akey=ca950945aa6aa851&timespan=1&type=1'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    try:
        return requests.get(url, headers=headers).text.strip()
    except:
        return None

# proxy_list=[get_proxy() for i in range(10)]
proxy_list=[None]

async def Crawling(start_page,end_page,sleep_time,level_list,proxy_list,url,headers,id):
    nums=0
    print('\n开始爬取'+str(id))
    with open(r'京东评论'+id+'_2_'+str(start_page)+'_'+str(end_page-1)+'.txt','wt',encoding='utf-8') as f:
        for scores in level_list:
            print('scores:',scores)
            for page in range(start_page,end_page):
                print('\rpage:',page,end='',flush=True) if page%5==0 else None
                data={ 'productId':id, 'score':str(scores), 'sortType':'5', 'page':str(page),'pageSize':'10','isShadowSku':'0','fold':'1'}
                try:
                    comments=requests.get(url=url,headers=headers,params=data,proxies={'http':random.choice(proxy_list)}).json()['comments']
                    if comments:
                        await asyncio.sleep(sleep_time)
                        for each in comments:
                            content=each['content']
                            score=each['score']
                            f.write(content+','+str(score)+'\n')
                            nums+=1
                    else:
                         print('\n'+id+'爬取完毕',end='\n\n')
                         break
                except:
                    print('Failed')
    with open(r'京东评论'+id+'_2_'+str(start_page)+'_'+str(end_page-1)+'.txt','r',encoding='utf-8') as f:
        data=f.readlines()
    lis=[]
    with open(r'京东评论'+id+'_'+str(start_page)+'_'+str(end_page-1)+'_'+str(nums)+'.txt','w',encoding='utf-8') as f:
        for each in data:
            if ','+str(1)  not in each and ','+str(2) not in each and ','+str(3) not in each   and ','+str(4) not in each and ','+str(5) not in each:
                lis.append(each.strip())
                continue
            else:
                lis.append(each.strip())
                for each2 in lis:
                    f.write(each2)
                f.write('\n')
                lis=[]
    os.remove(r'京东评论'+id+'_2_'+str(start_page)+'_'+str(end_page-1)+'.txt')

begin = time.time()
loop = asyncio.get_event_loop()
tasks = [loop.create_task(Crawling(start_page,end_page,sleep_time,level_list,proxy_list,url,headers,each)) for each in ids]
loop.run_until_complete(asyncio.wait(tasks))
end = time.time()
print(f'Total time: {end-begin} s')

开始爬取2372300
scores: 0
page: 0开始爬取100003233502
scores: 0
page: 0开始爬取100005011423
scores: 0
page: 0开始爬取62197285636
scores: 0
page: 10

KeyboardInterrupt: 

In [46]:
# 预处理数据集
import re
if not os.path.exists('预处理结果'):
    os.mkdir('预处理结果')
for pinpai in ['ALBD','AND','AT','BNL','GRN','LN','QPL','TB']:
    data = open('爬取结果/爬取结果_' + pinpai + '.txt', 'r', encoding='utf-8').readlines()
# for pinpai in ['1']:
#     data = open('4000条原始评论.txt', 'r', encoding='utf-8').readlines()
    print('\n', '开始预处理' + pinpai + '数据集', len(data), '条')
    data = list(set(data))
    for i in range(len(data)):
        # 删除空格
        data[i] = re.sub(r'\s+', '', data[i])
        # 删除特殊字符串
        data[i] = re.sub(r'[@#￥%……&*（）《》<>「」{}【】/\\]', ' ', data[i])
        # 删除数字
        data[i] = re.sub(r'\d+', '', data[i])
        # 删除英文
        data[i] = re.sub(r'[a-zA-Z]+', '', data[i])
        # 删除标点符号
        data[i] = re.sub(r'[，。！？、~;：;；“”‘’,.!?]', ' ', data[i])
        # 删除空行
        data[i] = data[i].strip()
        # 删除长度少于10的
        if len(data[i]) < 5:
            data[i] = ''
    print('预处理后' + pinpai + '数据集', len(data), '条')
    with open('预处理结果/预处理结果_' + pinpai + '.txt', 'w', encoding='utf-8') as f:
        for each in data:
            f.write(each + '\n') if each != '' else None


 开始预处理1数据集 4000 条
预处理后1数据集 3987 条
